In [1]:
import time
import win32api,win32gui,win32con, win32ui
from ctypes import *
import cv2
import sys
import os
from PIL import Image
import re
import pytesseract
import pyautogui
import numpy as np
import pyperclip
import json

In [2]:
VK_CODE = {'backspace':0x08,
           'tab':0x09,
           'clear':0x0C,
           'enter':0x0D,
           'shift':0x10,
           'ctrl':0x11,
           'alt':0x12,
           'pause':0x13,
           'caps_lock':0x14,
           'esc':0x1B,
           'spacebar':0x20,
           ' ':0x20,
           'page_up':0x21,
           'page_down':0x22,
           'end':0x23,
           'home':0x24,
           'left_arrow':0x25,
           'up_arrow':0x26,
           'right_arrow':0x27,
           'down_arrow':0x28,
           'select':0x29,
           'print':0x2A,
           'execute':0x2B,
           'print_screen':0x2C,
           'ins':0x2D,
           'del':0x2E,
           'help':0x2F,
           '0':0x30,
           '1':0x31,
           '2':0x32,
           '3':0x33,
           '4':0x34,
           '5':0x35,
           '6':0x36,
           '7':0x37,
           '8':0x38,
           '9':0x39,
           'a':0x41,
           'b':0x42,
           'c':0x43,
           'd':0x44,
           'e':0x45,
           'f':0x46,
           'g':0x47,
           'h':0x48,
           'i':0x49,
           'j':0x4A,
           'k':0x4B,
           'l':0x4C,
           'm':0x4D,
           'n':0x4E,
           'o':0x4F,
           'p':0x50,
           'q':0x51,
           'r':0x52,
           's':0x53,
           't':0x54,
           'u':0x55,
           'v':0x56,
           'w':0x57,
           'x':0x58,
           'y':0x59,
           'z':0x5A,
           'numpad_0':0x60,
           'numpad_1':0x61,
           'numpad_2':0x62,
           'numpad_3':0x63,
           'numpad_4':0x64,
           'numpad_5':0x65,
           'numpad_6':0x66,
           'numpad_7':0x67,
           'numpad_8':0x68,
           'numpad_9':0x69,
           'multiply_key':0x6A,
           'add_key':0x6B,
           'separator_key':0x6C,
           'subtract_key':0x6D,
           'decimal_key':0x6E,
           'divide_key':0x6F,
           'F1':0x70,
           'F2':0x71,
           'F3':0x72,
           'F4':0x73,
           'F5':0x74,
           'F6':0x75,
           'F7':0x76,
           'F8':0x77,
           'F9':0x78,
           'F10':0x79,
           'F11':0x7A,
           'F12':0x7B,
           'F13':0x7C,
           'F14':0x7D,
           'F15':0x7E,
           'F16':0x7F,
           'F17':0x80,
           'F18':0x81,
           'F19':0x82,
           'F20':0x83,
           'F21':0x84,
           'F22':0x85,
           'F23':0x86,
           'F24':0x87,
           'num_lock':0x90,
           'scroll_lock':0x91,
           'left_shift':0xA0,
           'right_shift ':0xA1,
           'left_control':0xA2,
           'right_control':0xA3,
           'left_menu':0xA4,
           'right_menu':0xA5,
           'browser_back':0xA6,
           'browser_forward':0xA7,
           'browser_refresh':0xA8,
           'browser_stop':0xA9,
           'browser_search':0xAA,
           'browser_favorites':0xAB,
           'browser_start_and_home':0xAC,
           'volume_mute':0xAD,
           'volume_Down':0xAE,
           'volume_up':0xAF,
           'next_track':0xB0,
           'previous_track':0xB1,
           'stop_media':0xB2,
           'play/pause_media':0xB3,
           'start_mail':0xB4,
           'select_media':0xB5,
           'start_application_1':0xB6,
           'start_application_2':0xB7,
           'attn_key':0xF6,
           'crsel_key':0xF7,
           'exsel_key':0xF8,
           'play_key':0xFA,
           'zoom_key':0xFB,
           'clear_key':0xFE,
           '+':0xBB,
           ',':0xBC,
           '-':0xBD,
           '.':0xBE,
           '/':0xBF,
           '`':0xC0,
           ';':0xBA,
           '[':0xDB,
           '\\':0xDC,
           ']':0xDD,
           "'":0xDE,
'`':0xC0}

In [3]:
def clickLeftCur():
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN|win32con.MOUSEEVENTF_LEFTUP, 0, 0)
def doubleclickLeftCur():
    clickLeftCur()
    clickLeftCur()

def moveCurPos(x,y):
    windll.user32.SetCursorPos(x, y)

def getCurPos():
    return win32gui.GetCursorPos()
    
def typeword(word):
    dict = {}
    #處理A~Z
    for w in range(ord('A'), ord('Z') + 1):
        dict[chr(w)] = w
        
    #處理0~9
    for w in range(ord('0'), ord('9') + 1):
        dict[chr(w)] = w
    
    #處理其他
    dict['/'] = 111
    dict['enter'] = 108
    dict[':'] = 0xBA
    for i in word:
        if i == ':':
            win32api.keybd_event(VK_CODE['left_shift'], 0,0,0)
            win32api.keybd_event(VK_CODE[';'], 0,0,0)
            time.sleep(.05)
            win32api.keybd_event(VK_CODE['left_shift'],0 ,win32con.KEYEVENTF_KEYUP ,0)
            win32api.keybd_event(VK_CODE[';'],0 ,win32con.KEYEVENTF_KEYUP ,0)
        else:
            win32api.keybd_event(VK_CODE[i],0,0,0)  
        time.sleep(0.2)
    

In [6]:
# #獲取滑鼠當前位置
xy=getCurPos()
print(xy)

(974, 533)


In [8]:
#virtual button (85,80)
kd_number = '#2006'
app_icon = (122, 1054)
zoom_out = (159, 942)
city_position = (950, 505)
crown = (1121, 307)
#judge = (482, 583)
duke = (793, 583)
# duke = (426-85, 587-80) #fake
architect = (1112, 583)
scientist = (1425, 590)
confirm = (966, 924)
chat_button = (281, 975)
chat_bar = (954, 1013)
search_coord = (684, 69)
search_kd = (708, 235)
search_x = (968, 240)
search_y = (1159, 242)
search_button = (1301, 228)
close_chat = (1366, 536)
home = zoom_out
chat_lt = (540, 83)
# chat_lt = (95, 83)
chat_rb = (1327, 978)
# chat_rb = (882, 978)
chat_window = chat_bar
chat_random = (974, 533)
#other variable
tesseract_path = 'C:/Program Files/Tesseract-OCR/tesseract'
crown_img_path = 'crown.png'
lt_for_check = (0, 0)
rb_for_check = (1274, 729)
time_limit1 = 60
time_limit2 = 120
mode = 'test'
last_saved_time = 0.0

In [9]:
def maintain_aspect_ratio_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the 0idth and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    return cv2.resize(image, dim, interpolation=inter)

def detect_icon(original_image, template):
    
    # Load template, convert to grayscale, perform canny edge detection
#     template = cv2.imread(icon_path)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    template = cv2.Canny(template, 50, 200)
    (tH, tW) = template.shape[:2]
    #cv2.imshow("template", template)

    # Load original image, convert to grayscale
#     original_image = cv2.imread(image_path)
    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    found = None

    # Dynamically rescale image for better template matching
    for scale in np.linspace(0.1, 3.0, 20)[::-1]:

        # Resize image to scale and keep track of ratio
        resized = maintain_aspect_ratio_resize(gray, width=int(gray.shape[1] * scale))
        r = gray.shape[1] / float(resized.shape[1])

        # Stop if template image size is larger than resized image
        if resized.shape[0] < tH or resized.shape[1] < tW:
            break

        # Detect edges in resized image and apply template matching
        canny = cv2.Canny(resized, 50, 200)
        detected = cv2.matchTemplate(canny, template, cv2.TM_CCOEFF)
        (_, max_val, _, max_loc) = cv2.minMaxLoc(detected)

        # Uncomment this section for visualization
        '''
        clone = np.dstack([canny, canny, canny])
        cv2.rectangle(clone, (max_loc[0], max_loc[1]), (max_loc[0] + tW, max_loc[1] + tH), (0,255,0), 2)
        cv2.imshow('visualize', clone)
        cv2.waitKey(0)
        '''

        # Keep track of correlation value
        # Higher correlation means better match
        if found is None or max_val > found[0]:
            found = (max_val, max_loc, r)

    # Compute coordinates of bounding box
    (_, max_loc, r) = found
    final_coord = (int(max_loc[0] * r), int(max_loc[1] * r))
    
    
    if final_coord != None:
        if final_coord[1] > 157:
            print(final_coord)
            return final_coord
    else:
        return final_coord
#     (start_x, start_y) = (int(max_loc[0] * r), int(max_loc[1] * r))
#     (end_x, end_y) = (int((max_loc[0] + tW) * r), int((max_loc[1] + tH) * r))

    # Draw bounding box on ROI
#     cv2.rectangle(original_image, (start_x, start_y), (end_x, end_y), (0,255,0), 2)
#     print(start_x, start_y)


def trim(origin, target):
    if target in origin:
        return (trim(origin[origin.index(target)+1:],target))
    else:
        return (origin)
    
def  action(kd, x,y,title,mode='run'):
    title_position = {'DUKE':duke, 'ARCH':architect, 'SCIE':scientist}


    

    moveCurPos(search_coord[0], search_coord[1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(search_kd[0], search_kd[1])
    clickLeftCur()
    time.sleep(1)
    
    typeword(kd)
    typeword(['enter'])
    
    moveCurPos(search_x[0], search_x[1])
    clickLeftCur()
    time.sleep(1)

    typeword(x)
    typeword(['enter'])

    moveCurPos(search_y[0], search_y[1])
    clickLeftCur()
    time.sleep(1)

    typeword(y)
    typeword(['enter'])

    moveCurPos(search_button[0], search_button[1])
    clickLeftCur()
    time.sleep(4)

    moveCurPos(city_position[0], city_position[1])
    clickLeftCur()
    time.sleep(1)

    
    #crown position depends
    img = pyautogui.screenshot(region=[lt_for_check[0],lt_for_check[1],rb_for_check[0]-lt_for_check[0],rb_for_check[1]-lt_for_check[1]]) # x,y,w,h
#     img = pyautogui.screenshot()
#     cv2.imwrite('middle.png',np.array(img))
#     img = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
    img_crown = cv2.imread(crown_img_path)
    tmp = detect_icon(np.array(img), img_crown)
    if tmp != None:
        crown = tmp
        moveCurPos(crown[0], crown[1])
        clickLeftCur()
        time.sleep(1)
    
    if mode == 'test':
        pass
    else:
        moveCurPos(title_position[title][0], title_position[title][1])
        clickLeftCur()
        time.sleep(1)

    moveCurPos(confirm[0], confirm[1])
    clickLeftCur()
    time.sleep(1)
#     moveCurPos(home[0], home[1])
#     clickLeftCur()
#     time.sleep(1)

#     moveCurPos(home[0], home[1])
#     clickLeftCur()
#     time.sleep(2)

    

def clear_lineup(line, clear_key):
    for key in clear_key:
        line[key] = []
    return line

def post2chat(text):
    pyperclip.copy(text)
    moveCurPos(chat_window[0], chat_window[1])
    clickLeftCur()
    time.sleep(1)

    win32api.keybd_event(VK_CODE['left_control'], 0,0,0)
    win32api.keybd_event(VK_CODE['v'], 0,0,0)
    time.sleep(.05)
    win32api.keybd_event(VK_CODE['left_control'],0 ,win32con.KEYEVENTF_KEYUP ,0)
    win32api.keybd_event(VK_CODE['v'],0 ,win32con.KEYEVENTF_KEYUP ,0)
    
    typeword(['enter'])
    moveCurPos(chat_random[0], chat_random[1])
    clickLeftCur()
    time.sleep(1)

def is_int(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

def remain_int(value):
    pass

In [18]:
#### nain function
if __name__ == '__main__':
    ##### Action
    moveCurPos(app_icon[0], app_icon[1])
    clickLeftCur()
    time.sleep(1)

    current_using_title = {'DUKE':{'coord':[], 'time':0}, 'ARCH':{'coord':[], 'time':0}, 'SCIE':{'coord':[], 'time':0}}
    lineup = {'DUKE':[], 'ARCH':[], 'SCIE':[]} #
    lineup_old = {'DUKE':[], 'ARCH':[], 'SCIE':[]} #
    
    while True:

        ##CHAT ROOM SCAN##

        ##1. screenshot
        img = pyautogui.screenshot(region=[chat_lt[0],chat_lt[1],chat_rb[0]-chat_lt[0],chat_rb[1]-chat_lt[1]]) # x,y,w,h
        img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)

        ##2. screenshot to text
        pytesseract.pytesseract.tesseract_cmd = tesseract_path
        for i in range(img.shape[0]):
            if np.array_equal(img[0-i][728],[229,117,14]):
                crop_y = 0-i
    #             print(crop_y)
                break
            else:
                crop_y = 0
        img = Image.fromarray(img[crop_y:,])
        text = pytesseract.image_to_string(img, lang='eng')
        #print(text)

        ##3. put text from screenshot to lineup
        #clear content over division line
        increase_num = 0
#         if 'LOAD' in text:
#             text = trim(text,'LOAD')
        print('1. Text after trim:\n',text)
#         li = list(filter(None, re.split(' |\n|]|\)', text)))
        li = list(filter(None, re.split(' |\n|]|\)|:|X|Y|x|y|¥|\.', text)))
        print(li)
#         continue
        for i in range(len(li)):
            if '#' in li[i]:
                if (li[i-1] == 'DUKE') or (li[i-1] == 'ARCH') or (li[i-1] == 'SCIE'):
                    #前置
                    check_end = 0
                    tmp = []
                    x_coord = ''
                    y_coord = ''
                    
                    #策略1: 找X&Y
#                     max_next = 4
#                     for k in range(max_next):
#                         if i+k+1 > len(li)-1:
#                             check_end = k
#                         else:
#                             check_end = max_next
#                     for k in range(check_end):
#                         tmp.append(li[i+k+1])
#                     for j in tmp:
#                         if 'X:' in j:
#                             x_coord = j[2:]
#                         if ('Y:' in j) or ('¥' in j) or ('y:' in j):
#                             if len(j) <= 2:
#                                 if (li[li.index(j)+1] == 'g') or li[li.index(j)+1] == '9g' or li[li.index(j)+1] == 'eg':
#                                     y_coord = li[li.index(j)+2]
#                                 else:
#                                     y_coord = li[li.index(j)+1]
#                             else:
#                                 y_coord = j[2:]
                                
                    #改策略: 井字號後出現的第一個數字屬於x, 第二個屬於Y
                    max_next = 4
                    for k in range(max_next):
                        if i+k+1 > len(li)-1:
                            check_end = k
                        else:
                            check_end = max_next+1
                    for k in range(check_end):
                        tmp.append(li[i+k])
                        
                    for j in tmp:
                        if is_int(j):
                            if len(x_coord)==0:
                                x_coord = j
                            else:
                                y_coord = j
                                
                                
                    
                    if current_using_title[li[i-1]]['time'] == - 1:
                        pass
                    else:
                        lineup[li[i-1]].append((li[i][2:], x_coord, y_coord))
                    increase_num += 1
            sisc = {'/D':
                        {'name':'DUKE',
                         'stop':"current_using_title['DUKE']['coord'] = [];current_using_title['DUKE']['time'] = -1;clear_lineup(lineup, ['DUKE'])",
                         'start':"current_using_title['DUKE']['time'] = 0"},
                    '/A':
                        {'name':'ARCH',
                         'stop':"current_using_title['ARCH']['coord'] = [];current_using_title['ARCH']['time'] = -1;clear_lineup(lineup, ['ARCH'])",
                         'start':"current_using_title['ARCH']['time'] = 0"},
                    '/S':
                       {'name':'SCIE',
                        'stop':"current_using_title['SCIE']['coord'] = [];current_using_title['SCIE'][time] = -1;clear_lineup(lineup, ['SCIE'])",
                        'start':"current_using_title['SCIE']['time'] = 0"},
                    '/X':
                       {'name':'ALL',
                        'stop':"current_using_title['DUKE']['coord'] = [];current_using_title['ARCH']['coord'] = [];current_using_title['SCIE']['coord'] = [];current_using_title['DUKE']['time'] = -1;current_using_title['ARCH']['time'] = -1;current_using_title['SCIE']['time'] = -1;clear_lineup(lineup, ['DUKE','ARCH','SCIE'])",
                        'start':"current_using_title['DUKE']['time'] = 0;current_using_title['ARCH']['time'] = 0;current_using_title['SCIE']['time'] = 0"}
                   }
            if '/' in li[i]:
                cmd_key = ''
                cmd_action = ''
                if len(li[i]) > 2:
                    cmd_key = li[i][:2]
                    cmd_action = li[i][2:]
                else:
                    cmd_key = li[i]
                    cmd_action = li[i+1]
                    
                if cmd_key in sisc.keys():
                    if cmd_action in sisc[cmd_key].keys():
                        exec(sisc[cmd_key][cmd_action])
                        #Respond to command
                        post2chat(cmd_action + ' ' + sisc[cmd_key]['name'].lower())
                        print(cmd_action + ' ' + sisc[cmd_key]['name'].lower())
        print('New lineup queue:',lineup)

        #Type division line##
        if increase_num > 0:
            moveCurPos(chat_window[0], chat_window[1])
            clickLeftCur()
            time.sleep(1)
    #         typeword('-- l o a d i n g --')
            
            #paste
            copy_text = 'Status: '
            for key in current_using_title.keys():
                if current_using_title[key]['time'] == -1:
                    copy_text += (' ' + key + ': N')
                else:
                    if len(current_using_title[key]['coord']) == 0:
                        copy_text += ('  ' + key + ':Y')
                    else:
                        copy_text += ('  ' + key + ':O')
            pyperclip.copy(copy_text)
            post2chat(copy_text)

        ##UPDATE CURRENT USING TITLE LIST##
        #case 1: times up (duke: 3m, arch & sci: 5m)
        for index in current_using_title.keys():
            if current_using_title[index]['time'] == -1:
                pass
            else:
                if index == 'DUKE':
                    time_limit = time_limit2
                else:
                    time_limit = time_limit1
                    
                if time.time() - current_using_title[index]['time'] >= time_limit:
                    current_using_title[index]['coord'] = []
                    current_using_title[index]['time'] = 0
                if len(current_using_title[index]['coord']) == 0:
                    print(index,'is available now !')
                else:
                    print('Left time of', index, ':', float(format(time.time()-current_using_title[index]['time'], '.2f')), '(s)')

        #case 2: done

        print('Current using:', current_using_title)



        ##EXECUTION GIVING TITLE## *GIVE ONE AT A TIME*
        for index in lineup.keys():
            if len(lineup[index]) != 0:
                if len(current_using_title[index]['coord']) == 0:
                    current_using_title[index]['coord'] = lineup[index].pop(0)
                    current_using_title[index]['time'] = float(format(time.time(), '.2f'))
                    
                    moveCurPos(close_chat[0], close_chat[1])
                    clickLeftCur()
                    time.sleep(1)
                    
                    action(current_using_title[index]['coord'][0],current_using_title[index]['coord'][1], current_using_title[index]['coord'][2],index,mode='run')
                    
                    moveCurPos(chat_button[0], chat_button[1])
                    clickLeftCur()
                    time.sleep(1)
                    
                    print('Giving title:', index, current_using_title[index]['coord'])
        print('===================================')
        if time.time() - last_saved_time > 60:
            moveCurPos(chat_random[0], chat_random[1])
            clickLeftCur()
            last_saved_time = time.time()
        
    

1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 't

1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 't

1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 't

1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 't

1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [RMO6] Nafro®

done thanks 4

['[RMO6', 'Nafro®', 'done', 'thanks', '4', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 't

...

1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
Left time of DUKE : 110.97 (s)
ARCH is available now !
Left time of SCIE : 29.15 (s)
Current using: {'DUKE': {'coord': ('126', '628', '166'), 'time': 1605879614.32}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
Left time of DUKE : 111.27 (s)
ARCH is available now !
Left time of SCIE : 29.45 (s)
Current using: {'DUKE': {'coord': ('126', '628', '166'), 'time': 1605879614.32}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New l

1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
Left time of DUKE : 116.37 (s)
ARCH is available now !
Left time of SCIE : 34.55 (s)
Current using: {'DUKE': {'coord': ('126', '628', '166'), 'time': 1605879614.32}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
Left time of DUKE : 116.69 (s)
ARCH is available now !
Left time of SCIE : 34.87 (s)
Current using: {'DUKE': {'coord': ('126', '628', '166'), 'time': 1605879614.32}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New l

1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
Left time of SCIE : 39.95 (s)
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
Left time of SCIE : 40.25 (s)
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
AR

1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
Left time of SCIE : 45.78 (s)
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
Left time of SCIE : 46.1 (s)
Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': ('126', '831', '96'), 'time': 1605879696.14}}
1. Text after trim:
 [RMO6] Banksia

scie done thank you

['[RMO6', 'Banksia', 'scie', 'done', 'thank', 'ou', '\x0c']
New lineup queue: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARC

KeyboardInterrupt: 

###### Section for testing

In [ ]:
count = 0

In [23]:
#Screenshot of crown
moveCurPos(app_icon[0], app_icon[1])
clickLeftCur()
time.sleep(1)
img = pyautogui.screenshot(region=[lt_for_check[0],lt_for_check[1],rb_for_check[0]-lt_for_check[0],rb_for_check[1]-lt_for_check[1]]) # x,y
img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
# cv2.imshow('img',img)
# cv2.waitKey(0)
cv2.imwrite(str(count)+'.png',img)
count+=1

In [41]:
def find_subarray(a,b):
    for i in range(a.shape[0]-b.shape[0]+1):
        for j in range(a.shape[1]-b.shape[1]+1):
#             if abs(np.linalg.norm(a[i:i+b.shape[0],j:j+b.shape[1]]) - np.linalg.norm(b)) < 0.002:
#                 return (i,j)
            if (i < 30) and (i > 20):
                if ((j > 20) and (j < 28)) or ((j > 728) and (j < 770)):
                    n4 = classify_hist_with_split(a[i:i+b.shape[0],j:j+b.shape[1]], b)
        #             print(type(n4))
                    if n4[0] > 0.9:
#                         print(n4[0])
                        return (i,j)
        #             if np.array_equal(a[i:i+b.shape[0],j:j+b.shape[1]], b):
        #                 #print('Found at index of (', i, ',', j, ')')
        #                 return (i,j)
    return crown

In [43]:
img = cv2.imread('7.png')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_crown = cv2.imread(crown_img_path)
# img_crown = cv2.cvtColor(img_crown, cv2.COLOR_BGR2GRAY)
# t = time.time()
print(find_subarray(img,img_crown))
# print(time.time()-t)
# print(img_crown)
# print('------------------------------')
# print(img[28:28+img_crown.shape[0],728:728+img_crown.shape[1]])
# for i in range(img_crown.shape[0]):
#     print()
#     print(img_crown[i])
#     print(img[28:28+img_crown.shape[0],728:728+img_crown.shape[1]][i])
#     print('---------------------------------')
# np.array_equal(img_crown, img[28:28+img_crown.shape[0],728:728+img_crown.shape[1]]) 25 21
        

0.9018283
(28, 22)


# Icon detection test

In [20]:
import cv2
import numpy as np

# Resizes a image and maintains aspect ratio
def maintain_aspect_ratio_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the 0idth and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    return cv2.resize(image, dim, interpolation=inter)

# Load template, convert to grayscale, perform canny edge detection
template = cv2.imread('tax.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template = cv2.Canny(template, 50, 200)
(tH, tW) = template.shape[:2]
cv2.imshow("template", template)

# Load original image, convert to grayscale
original_image = cv2.imread('tax_test.png')
gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
found = None

# Dynamically rescale image for better template matching
for scale in np.linspace(0.1, 3.0, 20)[::-1]:

    # Resize image to scale and keep track of ratio
    resized = maintain_aspect_ratio_resize(gray, width=int(gray.shape[1] * scale))
    r = gray.shape[1] / float(resized.shape[1])

    # Stop if template image size is larger than resized image
    if resized.shape[0] < tH or resized.shape[1] < tW:
        break

    # Detect edges in resized image and apply template matching
    canny = cv2.Canny(resized, 50, 200)
    detected = cv2.matchTemplate(canny, template, cv2.TM_CCOEFF)
    (_, max_val, _, max_loc) = cv2.minMaxLoc(detected)

    # Uncomment this section for visualization
    '''
    clone = np.dstack([canny, canny, canny])
    cv2.rectangle(clone, (max_loc[0], max_loc[1]), (max_loc[0] + tW, max_loc[1] + tH), (0,255,0), 2)
    cv2.imshow('visualize', clone)
    cv2.waitKey(0)
    '''

    # Keep track of correlation value
    # Higher correlation means better match
    if found is None or max_val > found[0]:
        found = (max_val, max_loc, r)

# Compute coordinates of bounding box
(_, max_loc, r) = found
(start_x, start_y) = (int(max_loc[0] * r), int(max_loc[1] * r))
(end_x, end_y) = (int((max_loc[0] + tW) * r), int((max_loc[1] + tH) * r))

# Draw bounding box on ROI
cv2.rectangle(original_image, (start_x, start_y), (end_x, end_y), (0,0,255), 2)
cv2.imshow('detected', original_image)
cv2.imwrite('detected.png', original_image)
cv2.waitKey(0)

-1

727 28
727 28


KeyboardInterrupt: 

In [42]:
def is_int(value):
    try:
        int(value)
        return True
    except ValueError:
        return False
is_int('123')

True